In [ ]:
import numpy as np
from pygel3d import graph
from pygel3d import hmesh, gl_display as gl
from pygel3d import jupyter_display as jd
import pygel3d as py3d
from plyfile import PlyData, PlyElement
import open3d as o3d
import open3d.core as o3c

## Generate some examples of meshes with previous method to build a mesh from skeleton

### This jupyter note book contains 4 baseline methods
- SQM method
- FEQ remesh method
- Skin Modifier in Blender
- Dual contouring based method

Our original data are `data/points.npy` which contains points' 3d coordinates, `data/edge_array.npy` which contains edge indexed array, `data/edge_radius.npy` which contains each edges radius, corresponding to the edge indexed array(stored by same order).

## Data Preparation

In this section we mainly handle the data. We define several functions to manipulate data.


Load `.npy` document and define a function write the numpy array data into a pygel3d-style `.obj` file. Note: this custom `.obj` file only for graph.

In [ ]:
points = np.load('../data/points.npy')
edges = np.load('../data/edge_array.npy')
radii = np.load('../data/edge_radius.npy')

# write a .obj file from the points, edges, and radii numpy arrays
def write_graph_obj(points, edges, radii, path):
    with open(path, 'w') as f:
        # f.write(f'g default\n')
        for point in points:
            f.write(f'n {point[0]} {point[1]} {point[2]}\n')
            
        # for edge in edges:
        #     f.write(f'l {edge[0]+1} {edge[1]+1}\n')

        for i, edge in enumerate(edges):
            radius = radii[i]
            # f.write(f'c {edge[0]} {edge[1]} {radius}\n')
            f.write(f'c {edge[0]} {edge[1]}\n')
            
        f.close()

- Define a function that use the edge radius array generate each vertex's radius, return an np array of vertex radius.
- Define a function that convert the pygel3d-style .obj file to standard .obj file that is readable for Blender.

In [ ]:
def edge_radius_to_vertex_radius(edges, edge_radius):
    vertex_radius = np.zeros((edges.max()+1,))
    for i, edge in enumerate(edges):
        vertex_radius[edge[0]] = max(edge_radius[i], vertex_radius[edge[0]])     
        vertex_radius[edge[1]] = max(edge_radius[i], vertex_radius[edge[1]])
    return vertex_radius


def convert_custom_obj_to_standard(input_file, output_file):
    with open(input_file, 'r') as input_obj:
        lines = input_obj.readlines()

    converted_lines = []

    for line in lines:
        if line.startswith('n '):  # Change 'n' to 'v' for vertex
            parts = line.strip().split()
            converted_line = f"v {' '.join(parts[1:])}\n"
            converted_lines.append(converted_line)
        elif line.startswith('c '):  # Change 'c' to 'l' for edges
            parts = line.strip().split()
            converted_line = f"l {' '.join(parts[1:])}\n"
            converted_lines.append(converted_line)

    # Reindex the vertex indices to start from 0
    vertex_index_offset = 0
    for line in converted_lines:
        if line.startswith('l '):
            parts = line.strip().split()
            converted_line = f"l {' '}"
            for v in parts[1:]:
                converted_line += f"{int(v)+1} "
            converted_line += '\n'
            converted_lines[vertex_index_offset] = converted_line
        vertex_index_offset += 1

    with open(output_file, 'w') as output_obj:
        output_obj.writelines(converted_lines)

## SQM method

In [ ]:
write_graph_obj(points, edges, radii, "../data/graph.obj")
g = graph.load("../data/graph.obj")
m = graph.to_mesh_cyl(g,1)
hmesh.save('../data/mesh_SQM.obj',m)

## FEQ method

In [ ]:
# # generate vertex radius from edge radius and save it to vertex_radius.npy
# vertex_radius = edge_radius_to_vertex_radius(edges, radii)/10
# np.save('vertex_radius.npy', vertex_radius)

# # generate feq mesh from vertex radius and save it to feq_mesh.obj
# FEQ_mesh = hmesh.skeleton_to_feq(g,vertex_radius)
# hmesh.save("../data/feq_mesh.obj",FEQ_mesh)

# # generate feq mesh from vertex radius with origin scale and save it to feq_mesh_scale1.obj
# FEQ_mesh_t1 = hmesh.skeleton_to_feq(g,vertex_radius*10)
# hmesh.save("../data/feq_mesh_scale1.obj",FEQ_mesh_t1)

## Duel contouring based method

- Generate mesh only using points information

In [ ]:
# create numpy array of shape (n, n, n) from list of indices
def voxel_grid_from_idx(indices):
    voxel_grid = np.zeros((indices.max(axis=0)+1))
    for index in indices:
        voxel_grid[tuple(index)] = 1
    return voxel_grid


# add points for each edges for each x distant of the edge length
def add_points_for_edges(points, edges, x=0.1):
    new_points = []
    for edge in edges:
        p1 = points[edge[0]]
        p2 = points[edge[1]]
        distance = np.sqrt(np.sum((p1 - p2)**2))
        num_points = int(distance / x)
        for i in range(num_points):
            new_points.append(p1 + (p2 - p1) * i / num_points)
    return np.concatenate((points, np.array(new_points)))

# cylinder function calculation using radius and height
def cylinder(radius, start_coord, end_coord, num_sample_cir, sample_interval_z):
    new_points = []
    # calculate the height of the cylinder
    height = np.sqrt(np.sum((start_coord - end_coord)**2))
    # calculate the slope of the cylinder
    slope = (end_coord - start_coord) / height
    # calculate the number of samples in z direction
    num_samples_z = int(height / sample_interval_z)
    # calculate the points for the cylinder
    for i in range(num_sample_cir):
        for j in range(num_samples_z):
            offset = start_coord + j * slope
            x = radius * np.cos(i * 2 * np.pi / num_sample_cir) + offset[0]
            y = radius * np.sin(i * 2 * np.pi / num_sample_cir) + offset[1]
            z = offset[2]
            new_points.append(np.array([x, y, z]))
    # calculate the points for the top and bottom of the cylinder
    for i in range(num_sample_cir):
        x = radius * np.cos(i * 2 * np.pi / num_sample_cir) + start_coord[0]
        y = radius * np.sin(i * 2 * np.pi / num_sample_cir) + start_coord[1]
        z = start_coord[2]
        new_points.append(np.array([x, y, z]))
        x = radius * np.cos(i * 2 * np.pi / num_sample_cir) + end_coord[0]
        y = radius * np.sin(i * 2 * np.pi / num_sample_cir) + end_coord[1]
        z = end_coord[2]
        new_points.append(np.array([x, y, z]))
    print(np.array(new_points).shape)
    return np.array(new_points)


# generate point cloud for edge cylinder, input:edges, points, edge radius, num_samples. output: points
def generate_point_cloud_for_edge_cylinder(edges, points, edge_radius, num_sample_cir, sample_interval_z):
    new_points = []

    for i, edge in enumerate(edges):
        p1 = points[edge[0]]
        p2 = points[edge[1]]
        new_points.append(cylinder(edge_radius[i], p1, p2, num_sample_cir, sample_interval_z))
    print(np.concatenate(new_points).shape)
    return np.concatenate((points, np.concatenate(new_points)))

        



In [ ]:
# generate point cloud
pcd = o3d.geometry.PointCloud()
# dot_graph = add_points_for_edges(points, edges, x=0.01)
dot_graph = generate_point_cloud_for_edge_cylinder(edges, points, radii, 8, 0.5)
pcd.points = o3d.utility.Vector3dVector(dot_graph)
o3d.io.write_point_cloud("../data/pcd.ply", pcd)

# load point cloud and create voxel grid
pcd_load = o3d.io.read_point_cloud("../data/pcd.ply")
pcd_load.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(len(points), 3)))
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_load, voxel_size=3)

voxels = voxel_grid.get_voxels()
indices = np.stack(list(vx.grid_index for vx in voxels))
print(indices)

voxel_grid = voxel_grid_from_idx(indices)
iso_mesh = 0
iso_mesh = hmesh.volumetric_isocontour(voxel_grid,high_is_inside=True,make_triangles=True,tau=0.8)
hmesh.save('../data/iso_mesh.obj', iso_mesh)


- First generate voxel grid from SQM mesh, then create the mesh from voxel grid.

In [ ]:
stick_mesh = hmesh.load("../data/mesh_SQM.obj")
hmesh.triangulate(stick_mesh)
hmesh.save("../data/mesh_SQM_tri.obj", stick_mesh)


In [ ]:
stick_mesh = o3d.io.read_triangle_mesh("../data/mesh_SQM_tri.obj")
voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(stick_mesh, 4)
o3d.io.write_voxel_grid("../data/voxel_grid.ply", voxel_grid)
voxels = voxel_grid.get_voxels()
indices = np.stack(list(vx.grid_index for vx in voxels))
print(indices)


voxel_grid = voxel_grid_from_idx(indices)
iso_mesh = 0
iso_mesh = hmesh.volumetric_isocontour(voxel_grid,high_is_inside=True,make_triangles=True,tau=0.85)
hmesh.save('../data/iso_mesh_from_stick.obj', iso_mesh)

In [ ]:
# skeleton = graph.LS_skeleton(g)
# graph.save('../data/skeleton.obj', skeleton)
# convert_custom_obj_to_standard('../data/skeleton.obj', '../data/skeleton_standard.obj')

## Isocontour method, using openvdb to store the grid.

In [1]:
# import the necessary packages
import pyopenvdb as vdb
import numpy as np
import matplotlib.pyplot as plt
import pygel3d as pyg
import scipy as sp
import cupy as cp


In [3]:
points = np.load('../data/points.npy')
edges = np.load('../data/edge_array.npy')
edge_radius = np.load('../data/edge_radius.npy')

### implement in openvdb

In [4]:
# define the function to generate the edge bounding box
def generate_edge_bounding_box(points, edges, edge_radius):
    edge_bounding_boxes = []
    for i in range(len(edges)):
        edge = edges[i]
        p0 = points[edge[0]]
        p1 = points[edge[1]]
        radius = edge_radius[i]
        q_0x = int(np.floor(min(p0[0], p1[0]) - radius))
        q_0y = int(np.floor(min(p0[1], p1[1]) - radius))
        q_0z = int(np.floor(min(p0[2], p1[2]) - radius))
        q_1x = int(np.ceil(max(p0[0], p1[0]) + radius))
        q_1y = int(np.ceil(max(p0[1], p1[1]) + radius))
        q_1z = int(np.ceil(max(p0[2], p1[2]) + radius))
        q_0 = np.array([q_0x, q_0y, q_0z])
        q_1 = np.array([q_1x, q_1y, q_1z])
        edge_bounding_boxes.append([q_0, q_1])
    return np.array(edge_bounding_boxes)

# scaling the data to enlarge the resolution
# the resolution of the grid is 2048x2048x2048
def scale_data(points, scale):
    points = points * scale
    return points

points = scale_data(points, 2)

# create voxel grid using pyopenvdb
# the resolution of the grid is 2048x2048x2048


In [5]:

vdb_grid = vdb.FloatGrid(0)
grid_accessor = vdb_grid.getAccessor()


In [7]:

# load points to voxel grid
def load_points_to_grid(points, grid):
    grid_accessor = grid.getAccessor()
    sum =0
    for point in points:
        sum += 1
        grid_accessor.setValueOn(point, 1.0)
# load_points_to_grid(points, grid_accessor)
# load edges bounding boxes to voxel grid
def load_edge_bounding_boxes_to_grid(edge_bounding_boxes, grid):
    grid_accessor = grid.getAccessor()
    for edge_bounding_box in edge_bounding_boxes:
        q_0 = edge_bounding_box[0]
        q_1 = edge_bounding_box[1]
        grid.fill(q_0, q_1, float(0))


# example_grid = vdb.FloatGrid(float('inf'))
# grid_accessor = vdb_grid.getAccessor()
# example_edge_bounding_boxes = generate_edge_bounding_box(example_points, example_edges, example_edge_radius)
# load_edge_bounding_boxes_to_grid(example_edge_bounding_boxes, example_grid)
edge_bounding_boxes = generate_edge_bounding_box(points, edges, edge_radius)
print(edge_bounding_boxes.shape)
load_edge_bounding_boxes_to_grid(edge_bounding_boxes, vdb_grid)
# print(example_edge_bounding_boxes.shape)

(50176, 2, 3)


In [8]:
vdb_grid.transform = vdb.createLinearTransform(voxelSize = 0.1)
print(edge_bounding_boxes.shape)
print(vdb_grid.activeVoxelCount())
(q0,q1) = vdb_grid.evalActiveVoxelBoundingBox()
print(q0,q1)

print(np.max(points, axis=0))

(50176, 2, 3)
264031737
(17, 22, 7) (1272, 1016, 1931)
[1270.87756348 1014.73309326 1929.93945312]


In [9]:
# calculate the distance of a point to a line segment
def distance_to_segment(p, p1, p2):
    v = p2 - p1
    w = p - p1
    c1 = np.dot(w, v)
    if c1 <= 0:
        return np.linalg.norm(p - p1)
    c2 = np.dot(v, v)
    if c2 <= c1:
        return np.linalg.norm(p - p2)
    b = c1 / c2
    pb = p1 + b * v
    return np.linalg.norm(p - pb)

    

from IPython.display import clear_output
# calculate the distance to the nearest edge inside each bounding box
# and store the corresponding bounding box index to a grid
# return a distance grid and a bounding box grid
def precompute_distances(points, edges, edge_radius, bounding_boxes, grid):
    tem_distance_grid = grid.deepCopy()
    print(tem_distance_grid.evalActiveVoxelBoundingBox())
    grid_accessor = tem_distance_grid.getAccessor()
    bounding_box_grid = grid.deepCopy()
    bounding_box_grid_accessor = bounding_box_grid.getAccessor()
    for i in range(len(edges)):
        edge = edges[i]
        p1 = points[edge[0]]
        p2 = points[edge[1]]
        bounding_box = bounding_boxes[i]
        clear_output(wait=True)
        for x in range(bounding_box[0][0], bounding_box[1][0]+1):
            for y in range(bounding_box[0][1], bounding_box[1][1]+1):
                for z in range(bounding_box[0][2], bounding_box[1][2]+1):
                    p = np.array([x, y, z])
                    if grid_accessor.isValueOn(p):
                        distance = distance_to_segment(p, p1, p2)
                        gaussian_distance = np.exp(-((distance/edge_radius[i])**2)*0.6931)
                        # print(gaussian_distance, grid_accessor.getValue(p))
                        if gaussian_distance > grid_accessor.getValue(p):
                            
                            grid_accessor.setValueOnly(p, gaussian_distance)
                            cur_index = i
                            bounding_box_grid_accessor.setValueOnly(p, cur_index)
                        else:
                            pass
                    else:
                        pass
        
        print(i, "/", len(edges))
    return tem_distance_grid, bounding_box_grid

    
distance_grid, bounding_box_grid = precompute_distances(points, edges,edge_radius, edge_bounding_boxes, vdb_grid)
# example_distance_grid, example_bounding_box_grid = precompute_distances(example_points, example_edges, example_edge_bounding_boxes, example_grid)


50175 / 50176


In [10]:
# vdb.write('../data/test.vdb', vdb_grid)

print(distance_grid.evalActiveVoxelBoundingBox())
print(bounding_box_grid.evalActiveVoxelBoundingBox())
print(distance_grid.evalMinMax())
print(bounding_box_grid.evalMinMax())

distance_grid.name = 'distance_grid'
bounding_box_grid.name = 'bounding_box_grid'
vdb.write('../data/distance_grid.vdb', distance_grid)
vdb.write('../data/bounding_box_grid.vdb', bounding_box_grid)

((17, 22, 7), (1272, 1016, 1931))
((17, 22, 7), (1272, 1016, 1931))
(0.0, 1.0)
(0.0, 50175.0)


In [18]:
# grid = vdb.read('../data/test.vdb', '')
# grid.name = 'test'
# grid_accessor = grid.getAccessor()
# print(grid.info())
# print(grid.evalActiveVoxelDim())


from IPython.display import clear_output
bounding_box_grid = vdb.read('../data/bounding_box_grid.vdb', 'bounding_box_grid')
distance_grid = vdb.read('../data/distance_grid.vdb', 'distance_grid')

# print(bounding_box_grid.evalMinMax())

# define a function to generate the 1D gaussian kernel
def gaussian_kernel_1d(size, center_coord, grid, omega, xmin, xmax, direction, radius):
    kernel = np.zeros(size)
    c = size // 2
    grid_accessor = grid.getAccessor()
    sum = 0
    if direction == 'x':
        d = 0
    elif direction == 'y':
        d = 1
    elif direction == 'z':
        d = 2
    else:
        raise ValueError('direction must be x, y or z')
    for i in range(size):
        coord = center_coord
        coord[d] += i - c
        if coord[d] < xmin[d]:
            coord[d] = xmin[d]
        if coord[d] > xmax[d]:
            coord[d] = xmax[d]
        disti = grid_accessor.getValue(coord)
        
        kernel[i] = np.exp(-omega * ((disti/radius)**2))
        sum += kernel[i]
    kernel = kernel / sum
    del grid_accessor
    return kernel


# define a function to do the discrete 1d gaussian convolution
def gaussian_convolution_1d(size, grid, node_coord, omega, xmin, xmax, direction, radius):
    if direction == 'x':
        d = 0
    elif direction == 'y':
        d = 1
    elif direction == 'z':
        d = 2
    else:
        raise ValueError('direction must be x, y or z')
    # temp_grid = grid.deepCopy()
    # grid_accessor = temp_grid.getAccessor()    
    # for i in range(xmax[0]-xmin[0]+1):
    #     for j in range(xmax[1]-xmin[1]+1):
    #         for k in range(xmax[2]-xmin[2]+1):
    #             center_coord = [xmin[0]+i, xmin[1]+j, xmin[2]+k]
    #             kernel = gaussian_kernel_1d(size, center_coord, grid, omega, xmin, xmax, direction, radius)
    #             new_value = 0
    #             for a in range(size):
    #                 offset = a - size // 2
    #                 cur_coord = center_coord + offset * np.array([d==0, d==1, d==2])
    #                 cur_value = grid_accessor.getValue(cur_coord)
    #                 new_value += cur_value * kernel[a]
    grid_accessor = grid.getAccessor()
    center_coord = node_coord
    kernel = gaussian_kernel_1d(size, center_coord, grid, omega, xmin, xmax, direction, radius)
    new_value = 0
    for a in range(size):
        offset = a - size // 2
        cur_coord = center_coord + offset * np.array([d==0, d==1, d==2])
        cur_value = grid_accessor.getValue(cur_coord)
        new_value += cur_value * kernel[a]
    del grid_accessor
    return new_value


# define a function to do the discrete 3d gaussian convolution
def gaussian_convolution_3d(size, grid,node_coord, omega, q0, q1, i, radius):
    x_result = gaussian_convolution_1d(size, grid, node_coord, omega, q0, q1, 'x', radius)
    y_result = gaussian_convolution_1d(size, grid, node_coord, omega, q0, q1, 'y', radius)
    z_result = gaussian_convolution_1d(size, grid, node_coord, omega, q0, q1, 'z', radius) 
    # print('edge', i, 'done')
    return x_result* y_result* z_result


# implement the 3d gaussian convolution on the whole skeleton
# parameters: edge_list, grid, bounding boxes, omega
def gaussian_convolution_skeleton(edges, distance_grid, bounding_box_grid, omega, bounding_boxes, edge_radii, kernel_size=5):
    new_grid = distance_grid.deepCopy()

    new_grid_accessor = new_grid.getAccessor()
    bounding_box_grid_accessor = bounding_box_grid.getAccessor()
    i = 0
    for edge,i in zip(edges, range(len(edges))):
        bounding_box = bounding_boxes[i]
        q0 = bounding_box[0]
        q1 = bounding_box[1]
        for x in range(q0[0],q1[0]):
            for y in range(q0[1], q1[1]):
                for z in range(q0[2], q1[2]):
                    coordinate = [x,y,z]
                    bounding_box_index = bounding_box_grid_accessor.getValue(coordinate)
                    bounding_box_index = int(bounding_box_index)
                    if bounding_box_index != i:
                        continue
                    new_value = gaussian_convolution_3d(kernel_size, 
                                                        new_grid, coordinate, omega, q0, q1, 
                                                        bounding_box_index, edge_radii[bounding_box_index])
                    new_grid_accessor.setValueOnly(coordinate, new_value)
        clear_output(wait=False)
        print(i, 'edges done', flush=True)
    return new_grid




new_grid = gaussian_convolution_skeleton(edges, distance_grid, 
                                         bounding_box_grid, 0.6931, 
                                         edge_bounding_boxes, edge_radius, 
                                         kernel_size=20)

# example_new_grid = gaussian_convolution_skeleton(example_edges, example_distance_grid,example_bounding_box_grid, 0.6931, example_edge_bounding_boxes, example_edge_radius, kernel_size=5)




4 edges done


KeyboardInterrupt: 

In [21]:
new_grid.name = 'isocontour2'
vdb.write('../data/isocontour2.vdb', new_grid)

NameError: name 'new_grid' is not defined

In [ ]:
vdb.write('../data/isocontour.vdb', new_grid)

In [ ]:
import pyopenvdb as vdb
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
grid = vdb.read('../data/isocontour2.vdb', 'isocontour2')
grid.name = 'isocontour2'
grid.background = 0.0
for node in grid.iterOnValues():
    if node.value == float('inf') or math.isnan(node.value):
        node.value = 0.0
print(grid.evalMinMax())
print(grid.info())


In [ ]:
vdb.write('../data/isocontour2.vdb', grid)

In [11]:
# select the voxels in grid with a specific isovalue 
# and deactivate all other voxels

def select_voxel(grid, isovalue):
    new_grid = grid.deepCopy()
    for node in new_grid.iterOnValues():
        value = node.value
        if value < isovalue+0.1 and value > isovalue-0.1:
            node.active = True
            node.value = 1
        else:
            node.active = False
    return new_grid

# new_grid = select_voxel(distance_grid, 0.5)




In [12]:
print(new_grid.evalMinMax())
new_grid.background = 0.0
print(new_grid.background)

(1.0, 1.0)
0.0


In [42]:

new_grid.name = 'new_grid'
vdb.write('../data/new_grid.vdb', new_grid, 'new_grid')

In [11]:
points, triangles, quads = distance_grid.convertToPolygons(isovalue=0.5, adaptivity=1)

In [12]:
print(points.shape)
print(quads.shape)
print(triangles.shape)
print(quads[1])
# using the points and quads generate a .obj file can be shown in blender
def generate_mesh_obj(points, triangles, quads):
    with open('../data/new_mesh.obj', 'w') as f:
        for p in points:
            f.write('v %f %f %f\n' % (p[0], p[1], p[2]))
        for face in quads:
            f.write('f %d %d %d %d\n' % (face[0]+1, face[1]+1, face[2]+1, face[3]+1))
        for triangle in triangles:
            f.write('f %d %d %d\n' % (triangle[0]+1, triangle[1]+1, triangle[2]+1))
        f.close()

generate_mesh_obj(points, triangles, quads)



(3855079, 3)
(3055555, 4)
(1599952, 3)
[6 1 0 5]


## implement convolution on a simple example skeleton

In [6]:
import pyopenvdb as vdb
import numpy as np
import matplotlib.pyplot as plt
import pygel3d as pyg
import scipy as sp
import cupy as cp

def generate_example_skeleton_data(num_points, num_edges):
    points = np.random.rand(num_points, 3) * 540
    edges = np.random.randint(0, num_points, (num_edges, 2))
    edge_radius = np.random.rand(num_edges)*50
    return points, edges, edge_radius

example_points, example_edges, example_edge_radius = generate_example_skeleton_data(10,10)

# define the function to generate the edge bounding box
def generate_edge_bounding_box(points, edges, edge_radius):
    edge_bounding_boxes = []
    for i in range(len(edges)):
        edge = edges[i]
        p0 = points[edge[0]]
        p1 = points[edge[1]]
        radius = edge_radius[i]
        q_0x = int(np.floor(min(p0[0], p1[0]) - radius))
        q_0y = int(np.floor(min(p0[1], p1[1]) - radius))
        q_0z = int(np.floor(min(p0[2], p1[2]) - radius))
        q_1x = int(np.ceil(max(p0[0], p1[0]) + radius))
        q_1y = int(np.ceil(max(p0[1], p1[1]) + radius))
        q_1z = int(np.ceil(max(p0[2], p1[2]) + radius))
        q_0 = np.array([q_0x, q_0y, q_0z])
        q_1 = np.array([q_1x, q_1y, q_1z])
        edge_bounding_boxes.append([q_0, q_1])
    return np.array(edge_bounding_boxes)

# scaling the data to enlarge the resolution
# the resolution of the grid is 2048x2048x2048
def scale_data(points, scale):
    points = points * scale
    return points

# points = scale_data(points, 1.5)

# create voxel grid using pyopenvdb
# the resolution of the grid is 2048x2048x2048

In [8]:
example_grid = vdb.FloatGrid(float('inf'))
grid_accessor = example_grid.getAccessor()

def load_edge_bounding_boxes_to_grid(edge_bounding_boxes, grid):
    grid_accessor = grid.getAccessor()
    for edge_bounding_box in edge_bounding_boxes:
        q_0 = edge_bounding_box[0]
        q_1 = edge_bounding_box[1]
        grid.fill(q_0, q_1, float('inf'))

example_edge_bounding_boxes = generate_edge_bounding_box(example_points, example_edges, example_edge_radius)
load_edge_bounding_boxes_to_grid(example_edge_bounding_boxes, example_grid)
print(example_edge_bounding_boxes.shape)


(10, 2, 3)
